In [ ]:
#default_exp experiments

# Experiments

> refactor modelling, experiment functionality

In [ ]:
#export
from torchtools.core import *
from torchtools.data import *
from torchtools.models import *
from torchtools.datasets import *
from torchtools.augmentations import *
#from torchtools.datablock import *
from torchtools.dataloader import *
from torchtools.configs import *

In [ ]:
#export
import pandas as pd
import numpy as np
from fastai.basics import *
#from fast_tabnet.core import *
from fastcore.script import *
from fastai.callback.tracker import *

In [ ]:
#export
from tsai.models.InceptionTimePlus import *
from tsai.models.TSTPlus import *
from tsai.models.utils import build_ts_model, transfer_weights

### Worklflow with Discret

### Data Setup

In [ ]:
#export
## data config
df_fn = 'bi_sample_anon.csv'
df_dir = Path('./data/custom')
df_path = Path(Path(df_dir)/df_fn)

trn_end = 120000
val_end = 160000
test_end = 200000
splits = (L(range(trn_end)), L(range(trn_end, val_end)))
df_config = f'{int(trn_end/1000)}_{int(val_end/1000)}_{int(test_end/1000)}'

col_config = '6chan_anon_discrete'
cols_c, cols_d, cols_y, n_train = get_discrete_config()

df_source = Path(df_fn).stem


dataset_name = f'{df_source}_{col_config}_{cols_y}_{df_config}'
data_params = defaultdict(lambda:None, {'df_fn':df_fn, 'df_dir':df_dir, 'df_path':df_path, 'trn_end':trn_end, 
                                        'val_end':val_end, 'splits':splits, 'col_config_id':col_config, 
                                        'cols_c':cols_c, 'cols_d':cols_d, 'cols_y':cols_y, 'ds_id':dataset_name})

In [ ]:
nrows=200000
df_main = pd.read_csv(data_params['df_path'], nrows=nrows)

# cols_c, cols_d, cols_y, n_train = get_discrete_config()
# splits = TSSplitter()(df_main)

In [ ]:
df_small = df_main.iloc[:50000]
splits_small = TSSplitter()(df_small)

In [ ]:
splits_small

(#2) [range(0, 40000),range(40000, 50000)]

In [ ]:
items = items_from_df(df_small, cols_c, cols_y, n_train, cols_d=cols_d)
Xc,Xd,y = items_to_arrays(items)

In [ ]:
items = items_from_df(df_small, cols_c, cols_y, n_train)
Xc,y = items_to_arrays(items)

In [ ]:
splits_test = L(L(range(7000)), L(range(7000,9000)), L(range(9000,10000)))

In [ ]:
splits_test

(#3) [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220

### Temp Exploration Reproducibility

In [ ]:
Xc.shape, y.shape, Xd.shape

((50000, 4, 10), (50000,), (50000, 2, 10))

In [ ]:
 dsets = TSDatasets3(X=Xc, X_dis=None, y=y, splits=splits_small)

In [ ]:
dsets[0]

(TSTensor([[  -7.8431,  100.0000,   -6.4977,  -24.5098,  -33.2226,  -42.7350,
           -55.2486,  -56.4972,  -30.6748,  100.0000],
         [ 100.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
             0.0000,    0.0000,    0.0000,    0.0000],
         [ -17.3010,  100.0000,  -30.6748,  -54.9451, -227.2727,  -27.8552,
          -400.0000, -147.0588, -100.0000,  -94.3396],
         [   0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
             0.0000,    0.0000,    0.0000,    0.0000]]),
 tensor(-56.4972))

In [ ]:
bs,verbose,shuffle_train=64,False,True
ss = TSStandardize(by_var=True, verbose=verbose)
#     augmix = AugmixSS()
ds = [dsets.subset(i) for i in range(dsets.n_subsets)]
dls = TSDataLoaders.from_dsets(*ds, bs=[bs]+[bs]*len(splits), batch_tfms=[ss], shuffle_train=shuffle_train)

setups torch.Size([40000, 4, 10]) False


KeyboardInterrupt: 

In [ ]:
xc,y = dls.one_batch()

In [ ]:
loss_fn = get_loss_fn('leaky_loss', alpha=0.5)

In [ ]:
torch.backends.cudnn.deterministic = True

In [ ]:
set_seed(1234)
dls.train.rng = random.Random(random.randint(0,2**32-1))
# bs,verbose,shuffle_train=64,False,True
# ss = TSStandardize(by_var=True, verbose=verbose)
# #     augmix = AugmixSS()
# ds = [dsets.subset(i) for i in range(dsets.n_subsets)]
# dls = TSDataLoaders.from_dsets(*ds, bs=[bs]+[bs]*len(splits), batch_tfms=[ss], shuffle_train=shuffle_train)
learn = Learner(dls, InceptionTimeSgm(4,1), wd=0.001, loss_func=loss_fn)
learn.fit_one_cycle(3)

In [ ]:
set_seed(1234)
dls.train.rng = random.Random(random.randint(0,2**32-1))
# bs,verbose,shuffle_train=64,False,True
# ss = TSStandardize(by_var=True, verbose=verbose)
# #     augmix = AugmixSS()
# ds = [dsets.subset(i) for i in range(dsets.n_subsets)]
# dls = TSDataLoaders.from_dsets(*ds, bs=[bs]+[bs]*len(splits), batch_tfms=[ss], shuffle_train=shuffle_train)
learn = Learner(dls, InceptionTimeSgm(4,1), wd=0.001, loss_func=loss_fn)
learn.fit_one_cycle(3)

In [ ]:
set_seed(1234)
dls.train.rng = random.Random(random.randint(0,2**32-1))
# bs,verbose,shuffle_train=64,False,True
# ss = TSStandardize(by_var=True, verbose=verbose)
# #     augmix = AugmixSS()
# ds = [dsets.subset(i) for i in range(dsets.n_subsets)]
# dls = TSDataLoaders.from_dsets(*ds, bs=[bs]+[bs]*len(splits), batch_tfms=[ss], shuffle_train=shuffle_train)
learn = Learner(dls, InceptionTimeSgm(4,1), wd=0.1, loss_func=loss_fn)
learn.fit_one_cycle(3)

In [ ]:
set_seed(1234)
dls.train.rng = random.Random(random.randint(0,2**32-1))
# bs,verbose,shuffle_train=64,False,True
# ss = TSStandardize(by_var=True, verbose=verbose)
# #     augmix = AugmixSS()
# ds = [dsets.subset(i) for i in range(dsets.n_subsets)]
# dls = TSDataLoaders.from_dsets(*ds, bs=[bs]+[bs]*len(splits), batch_tfms=[ss], shuffle_train=shuffle_train)
learn = Learner(dls, InceptionTimeSgm(4,1), wd=0.001, loss_func=loss_fn)
learn.fit_one_cycle(3, wd=0.1)

### Dataloaders Setup

In [ ]:
#export
def emb_sz_rule(n_cat):
    "Rule of thumb to pick embedding size corresponding to `n_cat`"
    return min(600, round(1.6 * n_cat**0.56))

def _one_emb_sz(classes, n, sz_dict=None):
    "Pick an embedding size for `n` depending on `classes` if not given in `sz_dict`."
    sz_dict = ifnone(sz_dict, {})
    n_cat = len(classes[n])
    sz = sz_dict.get(n, int(emb_sz_rule(n_cat)))  # rule of thumb
    return n_cat,sz

def get_emb_sz(to, sz_dict=None):
    "Get default embedding size from `TabularPreprocessor` `proc` or the ones in `sz_dict`"
    return [_one_emb_sz(to.classes, n, sz_dict) for n in to.cat_names]

def get_mod(dls, arch='inception', dropout=None, fc_dropout=None, pretrained=None):
    '''
    architectures:
    - inception
    - transformer
    - tst
    - inception_gb, transformer_gb
    - transformer_dl (for double loss, preds 1d, y 2d)
    - pretrained: specify pretrained model path, make sure it corresponds to the chosen architecture
    '''
    if dls.classification and not dls.mixed:
        model = InceptionTime(dls.n_channels, dls.c)
    
    elif arch=='inception_gb': #hack, works only for continuous channels and 1 target
        model = InceptionTime(dls.n_channels, 2)
    
    elif arch=='transformer_gb': #hack, works only for continuous channels and 1 target
        model = TST(dls.n_channels, 2, 10)
    
    elif arch=='transformer_dl': #hack, works only for continuous channels and exactly 2 targets with double_loss
        #return TST(dls.n_channels, 1, 10):
        model = TSTPlus(dls.n_channels, 1, seq_len=10, res_dropout=dropout, fc_dropout=fc_dropout, y_range=(-1,1))
    
    elif dls.n_channels==0:
        assert dls.cols_cat is not None or dls.cols_cont is not None, 'no tabular columns'
        emb_szs= [_one_emb_sz(dls.voc, c) for c in listify(dls.cols_cat)] 
        model = TabNetTT(emb_szs=emb_szs, n_cont=len(dls.cols_cont), out_sz=dls.n_targets)
    
    elif dls.mixed:
        emb_szs= [_one_emb_sz(dls.voc, c) for c in listify(dls.cols_cat)] 
        
        if dls.classification:
             model = InceptionTime_Mixed(dls.n_channels_c, dls.n_channels_d, dls.c, 
                                    len(dls.cols_cont), emb_szs=emb_szs)
        else:
            model = InceptionTimeD_Mixed(dls.n_channels_c, dls.n_channels_d, dls.n_targets, 
                                    len(dls.cols_cont), emb_szs=emb_szs)
    else:
        if dls.dataset.has_x[1]: ##discrete channels
            if arch=='transformer':
                model = TransformerSgmD(dls.n_channels, dls.n_targets, res_dropout=dropout)
            else:
                model = InceptionTimeD(dls.n_channels, dls.n_targets)
        else:
            if arch=='tst':
                #return TransformerSgm(dls.n_channels, dls.n_targets, res_dropout=dropout)
                model = TSTPlus(dls.n_channels, dls.n_targets, seq_len=10, res_dropout=dropout, y_range=(-1,1))
            if arch=='transformer':
                model = TransformerSgm(dls.n_channels, dls.n_targets, res_dropout=dropout)
            else:
                model = InceptionTimeSgm(dls.n_channels, dls.n_targets)
                
    if pretrained:
        transfer_weights(model, pretrained, exclude_head=True) #this works only for tsai plus models
    
    return model

In [ ]:
#export
def get_dls(df, cols_c, cols_y, splits, cols_d=None, bs=64, ds_type=TSDatasets5, shuffle_train=True,
           verbose=False, ss_dis=True, cols_cont=None, cols_cat=None, classification=False, stats=None):
    '''
    create dataloaders
    handling of discrete channels with cols_d and ss_dis
    NOTE: continuous tab cols 3d, cat tab cols 2d, legacy....
    '''
    
    items = items_from_df(df, cols_c, cols_y, len(splits[0]), cols_d=cols_d, tab_cols_c=cols_cont)
    ars=items_to_arrays(items)
    has_col=[cols_c is not None, cols_d is not None, cols_cont is not None]
    Xc, Xd, X_conts = map_xs(ars[:-1], has_col)
    
    y=ars[-1].astype(np.float)
    if classification:
        y, y_vocab = cats_from_df(df, listify(cols_y), len(splits[0]), add_na=False)
        y=y.squeeze()
        y=y.astype(np.long)
 
    if cols_cat is not None:
        X_cats, cat_maps = cats_from_df(df, cols_cat, len(splits[0]))
    else: X_cats, cat_maps = None, None

    _ytype=TensorCategory if classification else TensorFloat
    print(ds_type)
    dsets = ds_type(X_c=Xc, X_d=Xd, y=y, splits=splits, X_tcont=X_conts, X_tcat=X_cats, _ytype=_ytype)
#     dsets = ds_type(X=Xc, X_dis=Xd, y=y, splits=splits, X_tabc=X_conts, X_tabcat=X_cats, _ytype=_ytype)
    print(dsets.n_subsets)

    ##standardization: continuous channels always, discrete channels optional
    batch_tfms=[]
    print(has_col)
    ###!!!!HACK!!!!!
    if stats is not None: batch_tfms+=[TSStandardize(by_var=True, verbose=verbose).from_stats(*stats)]
    else:
        if has_col[0] and ss_dis: batch_tfms+=[TSStandardize(by_var=True, verbose=verbose)]
        if has_col[1] and ss_dis: batch_tfms+=[TSStandardize(by_var=True, verbose=verbose, discrete=True)]
#     augmix = AugmixSS()
#     print(batch_tfms)
#     return dsets
    ds = [dsets.subset(i) for i in range(dsets.n_subsets)]
#     return ds
#     dls = TSDataLoaders.from_dsets(*ds, bs=[bs]+[bs]*len(splits), batch_tfms=batch_tfms, shuffle_train=shuffle_train)
    dls = TSDataLoaders.from_dsets(*ds, bs=bs, batch_tfms=batch_tfms, shuffle_train=shuffle_train)
#     dls = TSDataLoaders.from_dsets(dsets.train, bs=[128,128])
    dls.n_channels = len(listify(cols_c)) + len(listify(cols_d))
    dls.n_channels_c = len(listify(cols_c)) 
    dls.n_channels_d = len(listify(cols_d))
    
    dls.n_targets = len(listify(cols_y))
    dls.cols_cat, dls.cols_cont = cols_cat, cols_cont
    if cols_cat is not None:
        dls.voc=cat_maps
    
    dls.mixed = dls.cols_cat is not None or dls.cols_cont is not None
        
    if classification:
        dls.y_vocab=y_vocab
        dls.c = len(dls.y_vocab[list(dls.y_vocab.keys())[0]])
    dls.classification = True if classification else False
    ##ToDO: for mixed input, store category info in dl
  
    return dls

In [ ]:
cols_cont=['x0_0', 'x0_1']

In [ ]:
items = items_from_df(df_small, cols_c, cols_y, len(splits[0]), cols_d=cols_d, tab_cols_c=cols_cont)
ars=items_to_arrays(items)
has_col=[cols_c is not None, cols_d is not None, cols_cont is not None]
Xc, Xd, X_conts = map_xs(ars[:-1], has_col)

In [ ]:
dsets=TSDatasets4(X=None, X_dis=None, y=y, X_tabc=X_conts)

NameError: name 'X' is not defined

In [ ]:
%%time
dls = get_dls(df_small, cols_c, cols_y, splits_test, cols_d=cols_d, ds_type=TSDatasets5)

<class 'torchtools.dataloader.TSDatasets5'>
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, <class 'torchtools.data.TensorFloat'>]
fast part
3
[True, True, False]
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, <class 'torchtools.data.TensorFloat'>]
fast part
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, <class 'torchtools.data.TensorFloat'>]
fast part
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, <class 'torchtools.data.TensorFloat'>]
fast part
setups torch.Size([7000, 4, 10]) False
setups torch.Size([7000, 2, 10]) True
CPU times: user 4.5 s, sys: 766 ms, total: 5.26 s
Wall time: 5.63 s


In [ ]:
TSDatasets4??

In [ ]:
%%time
dls = get_dls(df_small, None, cols_y, splits_test, cols_d=None, ds_type=TSDatasets4)

<class 'torchtools.dataloader.TSDatasets4'>
[None]
3
[None]
[None]
[None]


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 2)

In [ ]:
dls.n_subsets ## not set correctly with TSDatasets3

2

In [ ]:
dls[2]

In [ ]:
dls.one_batch()

(TSTensor([[[ 1.1964, -0.1871, -0.2048,  ...,  1.1964,  1.1964, -0.1482],
          [ 1.6677,  0.0114, -1.7891,  ...,  1.6677,  0.0114,  0.0114],
          [ 1.2029, -0.1595,  1.2029,  ..., -0.6687, -0.5053, -0.0933],
          [ 1.6715,  0.0149,  1.6715,  ...,  0.0149,  0.0149,  0.0149]],
 
         [[ 1.1964, -0.8428, -1.0945,  ..., -0.1015,  1.1964,  1.1964],
          [ 1.6677, -1.5513, -2.0853,  ...,  0.0114,  0.0114,  0.0114],
          [-0.3483, -1.0420, -0.6568,  ...,  1.2029,  1.2029, -0.8277],
          [-1.7474, -1.4910,  0.0149,  ...,  0.0149,  0.0149,  0.0149]],
 
         [[-0.1695, -0.6265, -0.7285,  ..., -0.4736,  1.1964, -0.4840],
          [ 1.6677, -1.6125, -1.9147,  ...,  0.0114,  0.0114,  0.0114],
          [-0.0517, -0.2326,  1.2029,  ..., -0.1295, -0.5932,  1.2029],
          [ 1.6715, -1.5779,  0.0149,  ..., -1.2496,  0.0149,  0.0149]],
 
         ...,
 
         [[-0.9986, -0.4819,  1.1964,  ..., -0.0808, -0.7478, -0.2011],
          [ 0.0114,  0.0114,  0.0114,

In [ ]:
%%time
dls = get_dls(df_main, cols_c, cols_y, splits, cols_d=cols_d, ds_type=TSDatasets3)

(200000, 4, 10)
(200000, 2, 10)
<class 'torchtools.dataloader.TSDatasets3'>
2
CPU times: user 8.39 s, sys: 995 ms, total: 9.38 s
Wall time: 8.02 s


In [ ]:
%%time
#TSDatasets2
dls = get_dls(df_main, cols_c, cols_y, splits, cols_d=cols_d, ds_type=TSDatasets2)

(200000, 4, 10) (200000, 2, 10)
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, None]
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, None]
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, None]
CPU times: user 1min 3s, sys: 508 ms, total: 1min 4s
Wall time: 1min 3s


### Basic Training

In [ ]:
#export
def _remove_augs(dls):
    '''
    remove augmentation transforms from dls.after_batch
    '''
    fs = [f for f in dls.train.after_batch.fs if not issubclass(type(f), AugTransform)]
    print(fs)
    dls.train.after_batch.fs.clear()
    for f in fs: dls.train.after_batch.add(f)
    #since fastai 2.29 dls.after_batch no longer automatically equal to dls.valid.after_batch etc..
    #for dl in dls:
    #    dl.after_batch=dls.after_batch
    print(dls.after_batch, dls.train.after_batch)

In [ ]:
import fastai
fastai.__version__

'2.3.1'

In [ ]:
dls = get_dls(df_small, data_params['cols_c'], data_params['cols_y'], splits_test, 
              cols_d=data_params['cols_d'])
dls.after_batch.add(Augmix())

<class 'torchtools.dataloader.TSDatasets5'>
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, <class 'torchtools.data.TensorFloat'>]
fast part
3
[True, True, False]
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, <class 'torchtools.data.TensorFloat'>]
fast part
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, <class 'torchtools.data.TensorFloat'>]
fast part
[<class 'torchtools.data.TSTensor'>, <class 'torchtools.data.TSIntTensor'>, <class 'torchtools.data.TensorFloat'>]
fast part
setups torch.Size([7000, 4, 10]) False
setups torch.Size([7000, 2, 10]) True


In [ ]:
for dl in dls:
    print(dl.after_batch)

Pipeline: TSStandardize -> TSStandardize -> Augmix
Pipeline: TSStandardize -> TSStandardize
Pipeline: TSStandardize -> TSStandardize


In [ ]:
ab = dls.after_batch

In [ ]:
ab.remove()

AttributeError: remove

In [ ]:
print(dls.after_batch, dls.train.after_batch, dls.valid.after_batch)
_remove_augs(dls)
print('remove')
print(dls.after_batch, dls.train.after_batch, dls.valid.after_batch)

Pipeline: TSStandardize -> TSStandardize Pipeline: TSStandardize -> TSStandardize Pipeline: TSStandardize -> TSStandardize
[TSStandardize:
encodes: (NumpyTensor,object) -> encodes
(TSTensor,object) -> encodes
(TSIntTensor,object) -> encodes
decodes: , TSStandardize:
encodes: (NumpyTensor,object) -> encodes
(TSTensor,object) -> encodes
(TSIntTensor,object) -> encodes
decodes: ]
Pipeline: TSStandardize -> TSStandardize Pipeline: TSStandardize -> TSStandardize
remove
Pipeline: TSStandardize -> TSStandardize Pipeline: TSStandardize -> TSStandardize Pipeline: TSStandardize -> TSStandardize


In [ ]:
#export
def run_training(dls, arch=None, seed=1234, n_epochs=None, max_lr=None, wd=None, 
                 loss_fn_name=None, alpha=None, metrics=unweighted_profit, 
                 N=2, magnitude=0.1, pct_start=0.3, div_factor=25.0, aug='randaugment', **kwargs):
    # model = ResNetSig(db.features, db.c).to(device)
    '''
    run a training cycle
    parameterization important for keeping track
    
    '''
    assert loss_fn_name and n_epochs, 'must pass loss_fn_name, and n_epochs'
    
    print(f'pct_start: {pct_start} div_factor: {div_factor}')
    set_seed(seed, reproducible=True)
#     model = arch(db.features, db.c)
#     model = arch(6,1)
    
    model = get_mod(self.dls, arch=arch)
    
    _remove_augs(dls)
    augs = RandAugment(N=N, magnitude=magnitude, verbose=True) if aug=='randaugment' else Augmix(
        N=N, magnitude=magnitude, verbose=True) if aug=='augmix' else None
#     augs  = Augmix(verbose=True)
    if augs: dls.after_batch.add(augs)       
    loss_fn = get_loss_fn(loss_fn_name, alpha=alpha) if not dls.classification else get_loss_fn
    print(loss_fn)
    
    learn = Learner(dls, model, loss_func=loss_fn, metrics=metrics)

    learn.fit_one_cycle(n_epochs, max_lr, wd=wd, pct_start=pct_start, div_factor=div_factor)
#     learn.recorder.plot_losses()
#     learn.recorder.plot_metrics()
    return learn

In [ ]:
#export
#train params
arch = InceptionTimeD
n_epochs = 5
max_lr = 1e-5
wd = 0.03
loss_fn_name = 'leaky_loss'
alpha = 0.5
metrics = [unweighted_profit]#, partial(unweighted_profit, threshold=0.2), 
           #partial(unweighted_profit, threshold=0.5)] #[weighted_profit, unweighted_profit_0, unweighted_profit_05]
N = 3
magnitude = 0.4
# bs = [64*4, 64*4*2]  treated as a data_param
# y_range = (-1, 1) # not sure yet about this one
seed = 1234
# ds_name = dataset_name #data_param inferred
# ds_path = str(ds_full_path) #data_param
pct_start=0.3                   #fastai default: 0.3
div_factor = 25.0               #fastai default 25.0
aug='augmix'

#default dict?
train_params = {'arch':arch, 'n_epochs':n_epochs, 'max_lr':max_lr, 'wd':wd, 'loss_fn_name':loss_fn_name, 'alpha':alpha,
               'metrics':metrics, 'N':N, 'magnitude':magnitude, 
                #'bs':bs, 
                'seed':seed, 
                #'ds_name':ds_name,
               'pct_start':pct_start, 'div_factor':div_factor, 'aug':aug}

In [ ]:
learn = run_training(dls, **train_params) 

pct_start: 0.3 div_factor: 25.0
[TSStandardize: (TSTensor,object) -> encodes
(NumpyTensor,object) -> encodes ]
Pipeline: TSStandardize Pipeline: TSStandardize
tfms None
functools.partial(<function leaky_loss at 0x7f872809f710>, alpha=0.5)


epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.173370,-0.063601,0.865342,00:06
1,-0.151190,-0.025506,0.824180,00:06
2,-0.206921,-0.041558,1.419818,00:06
3,-0.248992,-0.056973,1.623811,00:06
4,-0.276218,-0.062011,1.417221,00:06


weights [0.12069069 0.55244944 0.32685987], m 0.4964742723937071
ops in step 0 [TimestepZero: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 RandTimesteps: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timestepzero
randtimestep
yscale
ops in step 1 [RandomCrop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
randomcrop
yscale
ops in step 2 [TimeNormal: (TSTensor,object) -> encodes ]
timenormal torch.Size([128, 4, 10])
weights [0.09596037 0.3006876  0.60335203], m 0.9468840301872898
ops in step 0 [Zoomin: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
zoomin
ops in step 1 [TimeNormal: (TSTensor,object) -> encodes
 Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([128, 2, 10])
dimout
ops in step 2 [Maskout: (TSIntTensor,object) -> encodes
(TSTen

weights [0.55067066 0.00448835 0.44484099], m 0.7742702238921404
ops in step 0 [Cutout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
cutout
ops in step 1 [TimeNormal: (TSTensor,object) -> encodes
 RandTimesteps: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([128, 4, 10])
randtimestep
ops in step 2 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimestepZero: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
dimout
timestepzero
weights [0.12771486 0.04956191 0.82272323], m 0.4147685001727035
ops in step 0 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Crop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
crop
ops in step 1 [Maskout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
maskout
ops in step 2 [Crop: (TSIntTensor,object) -> encodes
(TSTensor,object) 

weights [0.21900226 0.31454934 0.4664484 ], m 0.1131809742168619
ops in step 0 [YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Zoomin: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ynormal
zoomin
dimout
ops in step 1 [RandTimesteps: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimestepZero: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeNormal: (TSTensor,object) -> encodes ]
randtimestep
timestepzero
timenormal torch.Size([128, 2, 10])
ops in step 2 [Maskout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
maskout
weights [0.37585757 0.32920988 0.29493255], m 0.8040871100129424
ops in step 0 [Zoomout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
zoomout
ywarp
timewarp
op

...

weights [0.7030395  0.21744442 0.07951608], m 0.12213668989378379
ops in step 0 [Cutout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Zoomout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
cutout
zoomout
timewarp
ops in step 1 [RandTimesteps: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 RandomCrop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
randtimestep
randomcrop
dimout
ops in step 2 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 RandomCrop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
randomcrop
dimout
weights [0.23688761 0.16729414 0.59581826], m 0.9310640202132504
ops in step 0 [RandTimesteps: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encode

weights [0.81140892 0.11707734 0.07151374], m 0.28084336793896425
ops in step 0 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Cutout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
dimout
cutout
ops in step 1 [RandTimesteps: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 RandZoom: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
randtimestep
randzoom
ops in step 2 [Zoomin: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
zoomin
timewarp
yscale
weights [0.30836694 0.65780713 0.03382593], m 0.3630809906124232
ops in step 0 [YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
yscale
ywarp
ops in step 

weights [0.45825607 0.25433241 0.28741152], m 0.2959771060259445
ops in step 0 [RandZoom: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
randzoom
ops in step 1 [Crop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
crop
ops in step 2 [RandTimesteps: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
randtimestep
weights [0.20328336 0.5359288  0.26078784], m 0.9838958917393326
ops in step 0 [RandTimesteps: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 RandomCrop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
randtimestep
ynormal
randomcrop
ops in step 1 [YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YScale: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimestepZero: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
ywarp
yscale
timestepzero
ops in step 2 [RandomCrop: (TSIntTensor,object) -> 

weights [0.08921797 0.76654062 0.1442414 ], m 0.6268887409515049
ops in step 0 [Zoomout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 TimestepZero: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
zoomout
dimout
timestepzero
ops in step 1 [TimeNormal: (TSTensor,object) -> encodes
 TimestepZero: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timenormal torch.Size([128, 4, 10])
timestepzero
dimout
ops in step 2 [Zoomin: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Zoomout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
zoomin
zoomout
weights [0.55410524 0.04125336 0.4046414 ], m 0.16083187007969527
ops in step 0 [Zoomout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
zoomout
ops in step 1 [TimeNormal: (TSTensor,object) -> encodes ]
timenormal torch.Size([

ops in step 1 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YNormal: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 YWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ynormal
ywarp
ops in step 2 [Maskout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
maskout
weights [0.26560446 0.2838539  0.45054164], m 0.6419169350425465
ops in step 0 [RandomCrop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Zoomout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
randomcrop
zoomout
ops in step 1 [TimeWarp: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
timewarp
ops in step 2 [Dimout: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes
 Crop: (TSIntTensor,object) -> encodes
(TSTensor,object) -> encodes ]
dimout
crop


In [ ]:
#export
def _losses_from_recorder(r, metrics=False):
    idx = slice(0,2) if not metrics else slice(2,None)
    return r.values[-1][idx]
def _minmax_values_from_recorder(r, metrics=False):
    idx = [0,1] if not metrics else list(range(2, len(r.values[0])))
    f = np.min if not metrics else np.max
    return L([f(L(r.values).itemgot(i)) for i in idx]) 

In [ ]:
#export
def get_recorder_dict(recorder):
    '''
    return a dictionary containing train and validation loss and metrics values
    '''
    metrics = recorder.metrics
#     loss_values = [recorder.losses[-1].item(), recorder.val_losses[-1].item()]
    loss_values = _losses_from_recorder(recorder)
#     loss_min_values = [np.min(recorder.losses), np.min(recorder.val_losses)] 
    loss_min_values = _minmax_values_from_recorder(recorder)
    metrics_values = _losses_from_recorder(recorder, metrics=True)
#     metrics_max_values = [np.max([m[i] for m in recorder.metrics]) for i in range(len(recorder.metrics[0]))]
    metrics_max_values = _minmax_values_from_recorder(recorder, metrics=True)
    recorder_keys = ['trn_loss', 'val_loss', 'trn_loss_min', 'val_loss_min', 
                     *[f'{m.name}_{i}_value' for i,m in enumerate(metrics)], *[f'{m.name}_{i}_max' for i,m in enumerate(metrics)]]
    return dict(zip(recorder_keys, loss_values+loss_min_values+metrics_values+metrics_max_values))

In [ ]:
rd = get_recorder_dict(learn.recorder)

In [ ]:
rd

{'trn_loss': -0.062010664492845535,
 'val_loss': 1.4172210693359375,
 'trn_loss_min': -0.062010664492845535,
 'val_loss_min': 1.4172210693359375}

In [ ]:
#export
def _to_flat_dict(train_params):
    flat_dict={}
    for key,value in train_params.items():
        if key=='metrics':
            for i,_ in enumerate(listify(value)):
                flat_dict[f'metric_{i}'] = value[i].__name__
        #arch parameter should be string, but used to be <class model>
        elif key=='arch' and not isinstance(value, str): flat_dict[key] = value.__name__
        else: flat_dict[key]=value
    return flat_dict

In [ ]:
#export
def _write_results(df, fn):
    if not os.path.isfile(fn):
        df.to_csv(fn, index=False)
    else:
        print('not new')
        df_old = pd.read_csv(fn)
        df_new = pd.concat([df_old, df], ignore_index=True, sort=False)
#         df.to_csv(fn, index=False, mode='a', header=False)
        df_new.to_csv(fn, index=False)

In [ ]:
train_params

{'arch': torchtools.models.InceptionTimeD,
 'n_epochs': 5,
 'max_lr': 1e-05,
 'wd': 0.03,
 'loss_fn_name': 'leaky_loss',
 'alpha': 0.5,
 'metrics': [<function torchtools.core.unweighted_profit(preds, y_true, threshold=0)>],
 'N': 3,
 'magnitude': 0.4,
 'seed': 1234,
 'pct_start': 0.3,
 'div_factor': 25.0,
 'aug': 'augmix'}

In [ ]:
_to_flat_dict(train_params)

{'arch': 'InceptionTimeD',
 'n_epochs': 5,
 'max_lr': 1e-05,
 'wd': 0.03,
 'loss_fn_name': 'leaky_loss',
 'alpha': 0.5,
 'metric_0': 'unweighted_profit',
 'N': 3,
 'magnitude': 0.4,
 'seed': 1234,
 'pct_start': 0.3,
 'div_factor': 25.0,
 'aug': 'augmix'}

In [ ]:
def run_experiment(dls, train_params, df_fn=None):
        '''
        could wrap the dataset parameters
        '''
        assert df_fn is not None, 'please specify results csv filename'
  
     
        learn = run_training(dls, **train_params)

        rec_dict = get_recorder_dict(learn.recorder)
        df_dict = dict()
        #     param_values = [arch.__name__, n_epochs, bs, seed, max_lr, wd, loss_fn_name, alpha, *[m.__name__ for m in metrics], N, magnitude]
        #     param_key = ['architecture', 'n_epochs', 'bs', 'seed', 'max_lr', 'wd', 'loss_fn', 'alpha', *[f'metric_{i}' for i,_ in enumerate(metrics)], 'N', 'magnitude']
        #     df_dict.update(train_params)
        df_dict.update(_to_flat_dict(train_params))
        df_dict.update(rec_dict)
        df_dict['Timestamp'] = str(datetime.now())
        ## store prediction in a separate file as tensors, but add filename
#         self._save_preds(test=True)
#         if self.save_model: self._save_model()
        _write_results(pd.DataFrame([df_dict], index=[0]), df_fn)
        return df_dict
    

In [ ]:
df_fn_test = Path('~/coding/python/betting/experiments/test_results.csv').expanduser()

In [ ]:
results_df = pd.read_csv(df_fn_test)

In [ ]:
results_df

,N,Timestamp,alpha,arch,aug,bs,div_factor,ds_name,loss_fn_name,magnitude,...,seed,trn_loss,trn_loss_min,unweighted_profit_0_max,unweighted_profit_0_value,val_loss,val_loss_min,val_preds,wd,ds_id
0,3,2020-05-21 14:36:42.977545,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,...,1234,-0.128333,-0.133325,1.912173,0.000000,-0.711629,-0.738477,NaN,0.03,NaN
1,3,2020-05-21 14:38:17.612064,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,...,1234,-0.181375,-0.181375,1.912173,0.000000,-0.683619,-0.740593,NaN,0.03,NaN
2,3,2020-05-21 16:19:49.366239,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,...,1234,-0.181424,-0.181424,1.912173,0.000000,-0.655837,-0.730417,NaN,0.03,NaN
3,3,2020-05-21 16:20:01.197565,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,...,1234,-0.204159,-0.204159,0.000000,0.000000,-0.671866,-0.711356,NaN,0.03,NaN
4,3,2020-05-21 16:20:12.393597,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,...,1234,-0.282855,-0.282855,0.044329,0.044329,-0.568804,-0.694458,NaN,0.03,NaN
5,3,2020-05-22 08:48:22.384761,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,...,1234,-0.222047,-0.222047,0.143756,-0.107899,-0.438311,-0.443605,NaN,0.03,NaN
6,3,2020-05-22 08:48:28.630306,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,...,1234,-0.327031,-0.327031,0.270344,0.055171,-0.427454,-0.453769,NaN,0.03,NaN
7,3,2020-05-22 08:48:34.883116,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,...,1234,-0.203794,-0.203794,0.040683,-0.045778,-0.443539,-0.443539,NaN,0.03,NaN
8,3,2020-05-22 09:45:35.273860,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_anon_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,...,1234,-0.336728,-0.351980,0.193756,-0.157899,-0.456215,-0.512078,NaN,0.03,NaN
9,3,2020-05-22 09:45:46.067508,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_anon_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,...,1234,-0.487051,-0.487051,0.253433,0.120056,-0.410117,-0.524226,NaN,0.03,NaN


In [ ]:
train_params

{'arch': torchtools.models.InceptionTimeD,
 'n_epochs': 5,
 'max_lr': 1e-05,
 'wd': 0.03,
 'loss_fn_name': 'leaky_loss',
 'alpha': 0.5,
 'metrics': [<function torchtools.core.unweighted_profit(preds, y_true, threshold=0)>],
 'N': 3,
 'magnitude': 0.4,
 'seed': 1234,
 'pct_start': 0.3,
 'div_factor': 25.0,
 'aug': 'augmix'}

### Wrapper

In [ ]:
df_path = Path('data/custom/bi_sample_anon.csv')

In [ ]:
#export
def _dict_product(params):
            values = list(itertools.product(*params.values()))
            return [dict(zip(params.keys(), values[i])) for i in range(len(values))]

In [ ]:
data_params.keys()

dict_keys(['df_fn', 'df_dir', 'df_path', 'trn_end', 'val_end', 'splits', 'col_config_id', 'cols_c', 'cols_d', 'cols_y', 'ds_id'])

minimal data configuation:
- path to source(dataframe)
- column configuration: explicit or implicit
- splits: explicit splits or function
- batch size

Experiments Configuration:
- data_params: dictionary, passed to ts_experiments.setup_data(df_path, columns, splits)
- train_params: dictionary, passed to ts_experiments.setup_training: training parameters
- (col_config: column configuration as a dictionary, passed to build_data_params)
- (build_data_params is a helper function to create basic data_params)
- batch_size: treated as data parameter, e.g. cannot be used as a grid search parameter

In [ ]:
#export
def _get_preds_fn(prefix='val'):
    return f'{prefix}_preds_{abs(hash(datetime.utcnow()))}.pt'

def _get_model_fn(prefix='model'):
    return f'{prefix}_{abs(hash(datetime.utcnow()))}' 

In [ ]:
#export
def _id_from_splits(splits):
    return '_'.join([(str((l[-1]+1)//1000)) for l in splits])
    
def _get_ds_id(data_params, splits):
    return f"{data_params['df_path'].stem}_{data_params['col_config_id']}_{_id_from_splits(splits)}"
      

In [ ]:
_get_ds_id(data_params, splits=splits_small)

'bi_sample_anon_6chan_anon_discrete_y0_7_9'

In [ ]:
ts_experiment = TSExperiments()

In [ ]:
tensor([1., 10., 1, .10, 1.])

tensor([ 1.0000, 10.0000,  1.0000,  0.1000,  1.0000])

In [ ]:
# #export
# def get_loss_fn_class(loss_fn_name, weight=None):
# #     weights = tensor([1., 10., 1, .10, 1.])
#     print(f'crosse entropy weigts {weight}')
#     return CrossEntropyLossFlat() if weight is None else CrossEntropyLossFlat(weight=weight.to(device))

In [ ]:
#export
class TSExperiments:
    '''
    Wrapper class for Timeseries modelling experiment
    needed: data_params, train_params for setup
    provides:
        - `run_experiment(df_results)`: run one modelling run using `train_params`
        - `grid_search(hypers, df_results)`: update `train_params` with each possible configuration of `hypers` 
        and run the respective experiment
    experimental results and all necessary parameters for reproducibility is stored in `df_results`
    
    '''
    def __init__(self, save_model=False, preds_path=None, model_path=None, results_path=None):
        ##reproducibility
        torch.backends.cudnn.deterministic = True
        
        #self.train_params = train_params #training params can change, e.g. when running grid search
        self.save_model = save_model## models are big
        self.preds_path = ifnone(preds_path, './experiments/preds')
        self.model_path = ifnone(model_path, './experiments/models')
        self.results_path = ifnone(results_path, './experiments/results')
        
    def setup_data(self, data_params):
        #read in dataframe
        self.data_params=data_params
        self.df_base = pd.read_csv(data_params['df_path'], nrows=data_params['nrows'])

        #get continuous, discrete, and dependent columns
        cols_c, cols_d, cols_y, splits, ss_dis = map(data_params.get, ['cols_c', 'cols_d', 'cols_y', 'splits', 'ss_dis'])
        cols_cat, cols_cont= map(data_params.get, ['cols_cat', 'cols_cont']) ## tabular data
        
        prune = data_params.get('prune', None)
        if prune is not None:
            assert prune in ['hcodds_col', 'overodds_col']
            prune_col = data_params.get(prune)
            print(prune_col)
            assert prune_col is not None, 'prune value has to be a valid columns'
            self.df_base.drop(self.df_base[self.df_base[prune_col]<=1].index, inplace=True)
            self.df_base.reset_index(inplace=True, drop=True)
        
        #get splits
#         print(splits, callable(splits))
        self.splits = splits(self.df_base) if callable(splits) else splits
#         print(self.splits)
        
        ##store some of the data parameters for later use
        self.bs = data_params['bs']
        self.ds_id = _get_ds_id(data_params, self.splits)
        self.classification = data_params.get('classification', False)
        self.prune = prune
        self.stats = data_params.get('stats')
#         self.dls = get_dls(self.df_base, cols_c, cols_y, self.splits, cols_d=cols_d, bs=self.bs, 
#                            ss_dis=ss_dis)
        self.dls = get_dls(self.df_base, cols_c, cols_y, self.splits, cols_d=cols_d, bs=self.bs, 
                           ss_dis=ss_dis, cols_cont=cols_cont, cols_cat=cols_cat, ds_type=TSDatasets5,
                          classification=self.classification, stats=self.stats)
        
        
    def setup_training(self, train_params):
        assert hasattr(self, 'data_params'), 'setup_data first'
        self.train_params = train_params
        self.train_params['bs']=self.bs
        self.train_params['ds_id'] = self.ds_id
        self.train_params['classification'] = self.classification
        self.train_params['prune'] = self.prune
        
        if self.train_params['classification']:
            assert self.train_params['loss_fn_name'] in ["crossentropy", "rww"]
    
 
    def _save_preds(self, test=False):
    #         val_preds_fn = _get_preds_fn()
        preds_fn = _get_preds_fn()
        preds, y_true = self.learn.get_preds(1)
        torch.save(preds, Path(self.preds_path)/preds_fn)
        self.df_dict.update({'val_preds':preds_fn})
        if len(list(self.dls))==3:
            preds_fn = _get_preds_fn('test')
            preds, y_true = self.learn.get_preds(2)
            torch.save(preds, Path(self.preds_path)/preds_fn)
            self.df_dict.update({'test_preds':preds_fn})
    
    def _save_model(self):
    #         val_preds_fn = _get_preds_fn()
        model_fn = self.model_fn
        self.learn.save(model_fn)
        self.df_dict.update({'model_fn':f'{self.learn.model_dir}/{model_fn}.pth'})

    
    
    def run_training(self, arch=None, seed=1234, n_epochs=None, max_lr=None, wd=None, 
                     loss_fn_name=None, alpha=None, metrics=unweighted_profit, 
                     N=2, magnitude=0.1, pct_start=0.3, div_factor=25.0, aug='randaugment', 
                     verbose=False, weight=None, save_best=False, **kwargs):
        # model = ResNetSig(db.features, db.c).to(device)
        '''
        run a training cycle
        parameterization important for keeping track
        '''
        assert loss_fn_name and n_epochs, 'must pass loss_fn_name, and n_epochs'

        print(f'pct_start: {pct_start} div_factor: {div_factor}')
       
        ## reset dls.rng --> consistent shuffling
        
#         huffle_fn(self, idxs): return self.rng.sample(idxs, len(idxs))
#         print(self.)
    #     model = arch(db.features, db.c)
       

        _remove_augs(self.dls)
        if aug=='randaugment':  augs=RandAugment(N=N, magnitude=magnitude, verbose=verbose)
#         elif aug=='augmix': augs=Augmix(N=N, magnitude=magnitude, verbose=verbose)
        elif aug=='augmix':
            _augmixtype=AugmixSS if kwargs.get('augmixss') is not None else Augmix
            augs=_augmixtype(N=N, magnitude=magnitude, verbose=verbose)
            print(f'augmix order {augs.order}')
        else:
            print(f'no augmentation with value {aug}')
            augs=None
        print(augs)
        print(augs is None)
        if augs: 
            self.dls.after_batch.add(augs)
            augs.setup(self.dls[0])
            ## Pipeline.add does not reorder the transforms, but we want the augmentation before the standardisation
            self.dls.after_batch.fs = self.dls.after_batch.fs.sorted(key='order')
        cbs = [SaveModelCallback(fname=f'{self.model_fn}_best_val'),
               #SaveModelCallback(fname=f'{self.model_fn}_best_combo_profit', monitor='combo_profit'),
              ] if save_best else None
            
#         loss_fn = get_loss_fn(loss_fn_name, alpha=alpha)
        loss_fn = get_loss_fn(loss_fn_name, alpha=alpha) if not self.dls.classification else get_loss_fn_class(
            loss_fn_name, weight=weight)
        print(loss_fn)
        
         
        set_seed(seed)
        self.dls.train.rng = random.Random(random.randint(0,2**32-1))
        pretrained = None
        if self.train_params.get('pretrained') is not None:
            pretrained = Path(self.model_path)/'pretrained'/self.train_params.get('pretrained')
        
#         model = arch(self.dls.n_channels, self.dls.n_targets)
        model = get_mod(self.dls, arch=self.train_params['arch'], dropout=self.train_params.get('dropout'),
                       fc_dropout=self.train_params.get('fc_dropout'), pretrained=pretrained)
        learn = Learner(self.dls, model, loss_func=loss_fn, metrics=metrics, model_dir=self.model_path,
                       wd=wd, cbs=cbs)
        print(learn.dls.after_batch)
        
#         print(f'wd: {wd} {learn.wd}')
        learn.fit_one_cycle(n_epochs, max_lr, wd=wd, pct_start=pct_start, div=div_factor)
#         learn.fit_one_cycle(n_epochs, max_lr, wd=wd)
    #     learn.recorder.plot_losses()
    #     learn.recorder.plot_metrics()
        return learn


    def run_experiment(self, df_fn=None):
        '''
        could wrap the dataset parameters
        '''
        assert df_fn is not None, 'please specify results csv filename'
        self.model_fn = _get_model_fn()
  
        self.learn = self.run_training(**self.train_params)
#         rec_dict = get_recorder_dict(self.learn.recorder)
        self.df_dict = dict()
        self.df_dict.update(_to_flat_dict(train_params))
        self.df_dict.update(get_recorder_dict(self.learn.recorder))
        self.df_dict['Timestamp'] = str(datetime.now())
        ## store prediction in a separate file as tensors, but add filename
        self._save_preds(test=False)
        if self.save_model: self._save_model()
        if self.save_model or self.train_params.get('save_best'):
            self.df_dict.update({'model_fn':f'{self.learn.model_dir}/{self.model_fn}.pth'})
        _write_results(pd.DataFrame([self.df_dict], index=[0]), Path(self.results_path)/df_fn)
#         return df_dict
    
    
   
    def run_grid_search(self, hypers:dict, df_results_fn=None):
        '''
        run hyper parameter grid search, note that this changes self.train_params
        '''
        hyper_configs = _dict_product(hypers) #list of dictionaries
        if hasattr(self, 'hyper_configs'): self.hyper_configs+=hyper_configs
        else: self.hyper_configs = hyper_configs
        for config in hyper_configs:
            self.train_params.update(config)
            print(self.train_params)
            self.run_experiment(df_fn=df_results_fn)

SyntaxError: invalid syntax (<ipython-input-8-4ff524dd8b01>, line 94)

In [ ]:
#export
def build_data_params(df_path, trn_end=None, val_end=None, test_end=None, splitter_fn=TSSplitter(), 
                      col_config=None, col_fn=None, bs=64, nrows=None, ss_dis=True, classification=False):
#     assert col_config or col_fn, 'need to pass either cont. cols and y cols, or a col_fn'

    assert col_config, 'need to pass columns configuration'
    
    if trn_end and val_end:
        splits=L(L(range(trn_end)), L(range(trn_end, val_end)))
        if test_end: splits.append(L(range(val_end, test_end)))
    else:
        splits = splitter_fn
    
    cols_c, cols_d, cols_y, cols_config_id, cols_cont, cols_cat, prune, hcodds_col = map(
        col_config.get, ['cols_c', 'cols_d', 'cols_y', 'id', 'cols_cont', 'cols_cat', 'prune', 'hcodds_col'])

#     dataset_name = f'{df
    
    data_params = defaultdict(lambda:None, {'df_path':df_path, 'splits':splits, 'col_config_id':cols_config_id, 
                                            'cols_c':cols_c, 'cols_d':cols_d, 'cols_y':cols_y, 'cols_cont':cols_cont,
                                             'cols_cat':cols_cat, 'hcodds_col': hcodds_col, 'bs':bs, 'prune':prune, 
                                            'nrows':nrows, 'ss_dis':ss_dis,'classification':classification})
#                'ds_full_path':ds_full_path, 
                 #'dataset_name':dataset_id, 
              
    return data_params

In [ ]:
## column configuration as dict, eventually move to separate configuration file
col_config = defaultdict(lambda:None)
col_config['cols_c'] = [[f'x{i}_{j}' for j in range(10)] for i in [0,1,3,4]]
col_config['cols_d'] = [[f'x{i}_{j}' for j in range(10)] for i in [2,5]]
col_config['cols_y']= 'y0'
col_config['id']='anon10hc_4c_2d_y'  ## put it all in one config file with this unique identifier

In [ ]:
data_params = build_data_params(df_path, cols_config=col_config, nrows=10000, trn_end=5000, val_end=6000)

In [ ]:
data_params['col_config_id'], data_params['bs']

('anon10hc_4c_2d_y', 64)

In [ ]:
ts_experiment = TSExperiments()

In [ ]:
ts_experiment.setup_data(data_params)

(10000, 4, 10) (10000, 2, 10)
<class '__main__.TSDatasets3'>
2
torch.Size([5000, 4, 10]) torch.Size([5000]) torch.Size([5000, 2, 10])
torch.Size([1000, 4, 10]) torch.Size([1000]) torch.Size([1000, 2, 10])


In [ ]:
hypers = {'max_lr':[3e-5], 'n_epochs':[7], 'N':[3], 'magnitude':[0.4]}

In [ ]:
train_params

{'arch': torchtools.models.InceptionTimeD,
 'n_epochs': 7,
 'max_lr': 3e-05,
 'wd': 0.03,
 'loss_fn_name': 'leaky_loss',
 'alpha': 0.5,
 'metrics': [<function torchtools.core.unweighted_profit(preds, y_true, threshold=0)>],
 'N': 3,
 'magnitude': 0.4,
 'seed': 1234,
 'pct_start': 0.3,
 'div_factor': 25.0,
 'aug': 'augmix',
 'ds_id': 'bi_sample_anon_None_y0_4_5',
 'bs': None}

In [ ]:
ts_experiment.setup_training(train_params)

In [ ]:
ts_experiment.run_grid_search(hypers, df_fn_test)

{'arch': <class 'torchtools.models.InceptionTimeD'>, 'n_epochs': 7, 'max_lr': 3e-05, 'wd': 0.03, 'loss_fn_name': 'leaky_loss', 'alpha': 0.5, 'metrics': [<function unweighted_profit at 0x7f2700319d40>], 'N': 3, 'magnitude': 0.4, 'seed': 1234, 'pct_start': 0.3, 'div_factor': 25.0, 'aug': 'augmix', 'ds_id': 'bi_sample_anon_anon10hc_4c_2d_y_y0_4_5', 'bs': 64}
pct_start: 0.3 div_factor: 25.0
[TSStandardize: (TSTensor,object) -> encodes
(NumpyTensor,object) -> encodes ]
Pipeline: TSStandardize Pipeline: TSStandardize
augmix
tfms None
False
functools.partial(<function leaky_loss at 0x7f2700319830>, alpha=0.5)
Pipeline: TSStandardize -> Augmix


epoch,train_loss,valid_loss,unweighted_profit,time
0,-0.145643,0.037035,-0.122274,00:03
1,-0.192670,0.001495,1.298082,00:04
2,-0.246716,-0.069481,1.126039,00:04
3,-0.461523,0.141900,-0.311868,00:04
4,-0.690655,0.233454,0.600721,00:04
5,-0.871654,0.233166,0.900510,00:03
6,-1.131848,0.293372,0.009376,00:03


not new


In [ ]:
results_df = pd.read_csv(df_fn_test)

In [ ]:
pd.options.display.max_columns = 99
results_df

,N,Timestamp,alpha,arch,aug,bs,div_factor,ds_name,loss_fn_name,magnitude,max_lr,metric_0,n_epochs,pct_start,seed,trn_loss,trn_loss_min,unweighted_profit_0_max,unweighted_profit_0_value,val_loss,val_loss_min,val_preds,wd,ds_id
0,3,2020-05-21 14:36:42.977545,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,0.00001,unweighted_profit,2,0.3,1234,-0.128333,-0.133325,1.912173,0.000000,-0.711629,-0.738477,NaN,0.03,NaN
1,3,2020-05-21 14:38:17.612064,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,0.00001,unweighted_profit,5,0.3,1234,-0.181375,-0.181375,1.912173,0.000000,-0.683619,-0.740593,NaN,0.03,NaN
2,3,2020-05-21 16:19:49.366239,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,0.00003,unweighted_profit,3,0.3,1234,-0.181424,-0.181424,1.912173,0.000000,-0.655837,-0.730417,NaN,0.03,NaN
3,3,2020-05-21 16:20:01.197565,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,0.00010,unweighted_profit,3,0.3,1234,-0.204159,-0.204159,0.000000,0.000000,-0.671866,-0.711356,NaN,0.03,NaN
4,3,2020-05-21 16:20:12.393597,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,0.00030,unweighted_profit,3,0.3,1234,-0.282855,-0.282855,0.044329,0.044329,-0.568804,-0.694458,NaN,0.03,NaN
5,3,2020-05-22 08:48:22.384761,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,0.00003,unweighted_profit,4,0.3,1234,-0.222047,-0.222047,0.143756,-0.107899,-0.438311,-0.443605,NaN,0.03,NaN
6,3,2020-05-22 08:48:28.630306,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,0.00010,unweighted_profit,4,0.3,1234,-0.327031,-0.327031,0.270344,0.055171,-0.427454,-0.453769,NaN,0.03,NaN
7,3,2020-05-22 08:48:34.883116,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,0.00030,unweighted_profit,4,0.3,1234,-0.203794,-0.203794,0.040683,-0.045778,-0.443539,-0.443539,NaN,0.03,NaN
8,3,2020-05-22 09:45:35.273860,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_anon_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,0.00003,unweighted_profit,7,0.3,1234,-0.336728,-0.351980,0.193756,-0.157899,-0.456215,-0.512078,NaN,0.03,NaN
9,3,2020-05-22 09:45:46.067508,0.5,InceptionTimeD,augmix,"[256, 512]",25.0,bi_sample_anon_6chan_anon_discrete_y0_120_160_200,leaky_loss,0.4,0.00010,unweighted_profit,7,0.3,1234,-0.487051,-0.487051,0.253433,0.120056,-0.410117,-0.524226,NaN,0.03,NaN


In [ ]:
ts_experiment.train_params['cols_config_id']

KeyError: 'cols_config_id'

### CLI

In [ ]:
#export
def _get_arch(arch:str, with_discrete=False):
    if arch.lower()=='inception': return InceptionTimeSgm if not with_discrete else InceptionTimeD
    elif arch.lower()=='resnet': return 'ResNet not implemented'
    else: return None

In [ ]:
COL_CONFIG = 'config2.json'

@call_parse
def main(n_epochs:Param(help="n_epochs list", nargs='+', type=int)=[10],
         max_lr:Param(help="max_lr list", nargs='+', type=float)=[1e-5],
         wd:Param(help="wd (weight decay): hyperparameter list of floats", nargs='+', type=float)=[0.03],
         div_factor:Param(help="div_factor hyperparameter list", nargs='+', type=float)=[25.0],
         seed:Param(help="seed hyperparameter list", nargs='+', type=int)=[1234],
         N:Param(help="N hyperparameter list", nargs='+', type=int)=[3],
         magnitude:Param(help="augmentation magnitude: hyperparameter list of floats", nargs='+', type=float)=[0.3],
         alpha:Param(help="alpha hyperparameter list", nargs='+', type=float)=[0.5],
         aug:Param(help="augmentation policy", choices=[None, 'randaugment', 'augmix'], type=str)=None,
         nrows:Param(help="n_epochs list", type=int)=None,
         bs:Param(help="batch size", type=int)=128,
         trn_end:Param(help="n_epochs list", type=int)=None,
         val_end:Param(help="n_epochs list", type=int)=None,
         test_end:Param(help="n_epochs list", type=int)=None,
         df_fn:Param(help="dataframe filename", type=str)='bi_sample_anon.csv',         
         df_dir:Param(help="dataframe dir", type=str)='./data/custom',
         df_results:Param(help="results dataframe filename", type=str)='results_script.csv',
         config_fn:Param(help="json column configuration filename", type=str)=COL_CONFIG,    
         config_id:Param(help="column configuration id", type=str)='anon2hc_4c_2d_y',
         arch:Param(help="model architecture", choices=['inception', 'resnet'], type=str)='inception',
         upper:Param("Convert to uppercase?", bool_arg)=False):
#     print(msg.upper() if upper else msg)
    
   
    
    train_params['aug']=aug
    
    df_path=Path(df_dir)/df_fn
    print(df_path)
    
    col_config=read_config(config_id, config_fn)
    data_params = build_data_params(df_path, col_config=col_config, nrows=nrows, trn_end=trn_end, val_end=val_end,
                                   test_end=test_end, bs=bs)
#     print(data_params)

    train_params['metrics']=[unweighted_profit, unweighted_profit_05]
    train_params['arch']=_get_arch(arch, col_config['cols_d'] is not None)
    ts_experiment = TSExperiments()
    ts_experiment.setup_data(data_params)
    ts_experiment.setup_training(train_params)
                                 
    hypers = {'n_epochs': n_epochs, 'max_lr':max_lr, 'wd':wd, 'seed': seed, 'div_factor':div_factor,
             'N':N, 'magnitude':magnitude}
    print(hypers)
    
    ts_experiment.run_grid_search(hypers, df_results)